### Leads Scoring Model

#### -- Import Statements --

In [1]:
# All Imports
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

############### Write code below ##################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pymysql

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas_profiling 

import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')

from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

## All import statements here --

def disp_missing_val(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
    missing_value_df.plot(kind='bar')
    
def pct2float(x):
    return float(x.strip('%'))/100
    
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'    
    
def mysql_db_conn():
    host = "i2rds-read-replica2.cuoivis0lb68.us-east-1.rds.amazonaws.com"
    port = 3306
    user = "product"
    password = "pr0ducT3"
    database = "melv1n_crm2" 
    
    conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')
    
    return conn  

def do_data_profiling(df, filename):
    '''
    Function to do basic data profiling
    Required Input - 
        - df = Pandas DataFrame
        - filename = Path for output file with a .html extension
    Expected Output -
        - HTML file with data profiling summary
    '''
    profile = pandas_profiling.ProfileReport(df)
    profile.to_file(outputfile = filename)
    print("Data profiling done")
    
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

def optimize_df_mem(df):
    # select only categorical fields
    gl_obj = df.select_dtypes(include=['object']).copy()    
    converted_obj = pd.DataFrame()
    
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
        """if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]"""
        
    print("%s memory usage before optimization: %s"%(df.name,mem_usage(gl_obj)))
    print("%s memory usage after optimization: %s"%(df.name,mem_usage(converted_obj)))
    print("      ***       ")
          
    return converted_obj    

def mysql_db(sql):    
    conn = mysql_db_conn()
    df = pd.read_sql_query(sql,conn)
    return df

def mysql_db_large(db,tbl,columns,limit,chunks=10000):
    
    query = "select "+columns+ " from "+db+"."+tbl+ " limit "+limit+";"
    splits = []
    conn = mysql_db_conn()
    for chunk in pd.read_sql(query,conn,chunksize=chunks):
        splits.append(chunk)
        concat_df  = pd.concat(splits,ignore_index=True)
    return concat_df

## Data preprocessing helper methods --

def drop_allsame(df):
    '''
    Function to remove any columns which have same value all across
    Required Input - 
        - df = Pandas DataFrame
    Expected Output -
        - Pandas dataframe with dropped no variation columns
    '''
    to_drop = list()
    for i in df.columns:
        if len(df.loc[:,i].unique()) == 1:
            to_drop.append(i)
    return df.drop(to_drop,axis =1)

def treat_missing_numeric(df,columns,how = 'mean'):
    '''
    Function to treat missing values in numeric columns
    Required Input - 
        - df = Pandas DataFrame
        - columns = List input of all the columns need to be imputed
        - how = valid values are 'mean', 'mode', 'median','ffill', numeric value
    Expected Output -
        - Pandas dataframe with imputed missing value in mentioned columns
    '''
    if how == 'mean':
        for i in columns:
            print("Filling missing values with mean for columns - {0}".format(i))
            df.ix[:,i] = df.ix[:,i].fillna(df.ix[:,i].mean())
            
    elif how == 'mode':
        for i in columns:
            print("Filling missing values with mode for columns - {0}".format(i))
            df.ix[:,i] = df.ix[:,i].fillna(df.ix[:,i].mode())
    
    elif how == 'median':
        for i in columns:
            print("Filling missing values with median for columns - {0}".format(i))
            df.ix[:,i] = df.ix[:,i].fillna(df.ix[:,i].median())
    
    elif how == 'ffill':
        for i in columns:
            print("Filling missing values with forward fill for columns - {0}".format(i))
            df.ix[:,i] = df.ix[:,i].fillna(method ='ffill')
    
    elif type(how) == int or type(how) == float:
        for i in columns:
            print("Filling missing values with {0} for columns - {1}".format(how,i))
            df.ix[:,i] = df.ix[:,i].fillna(how)
    else:
        print("Missing value fill cannot be completed")
    return df

def treat_missing_categorical(df,columns,how = 'mode'):
    '''
    Function to treat missing values in categorical columns
    Required Input - 
        - df = Pandas DataFrame
        - columns = List input of all the columns need to be imputed
        - how = valid values are 'mode', any string or numeric value
    Expected Output -
        - Pandas dataframe with imputed missing value in mentioned columns
    '''
    if how == 'mode':
        for i in columns:
            print("Filling missing values with mode for columns - {0}".format(i))
            df.ix[:,i] = df.ix[:,i].fillna(df.ix[:,i].mode()[0])
    elif type(how) == str:
        for i in columns:
            print("Filling missing values with {0} for columns - {1}".format(how,i))
            df.ix[:,i] = df.ix[:,i].fillna(how)
    elif type(how) == int or type(how) == float:
        for i in columns:
            print("Filling missing values with {0} for columns - {1}".format(how,i))
            df.ix[:,i] = df.ix[:,i].fillna(str(how))
    else:
        print("Missing value fill cannot be completed")
    return df
    
def min_max_scaler(df,columns):
    '''
    Function to do Min-Max scaling
    Required Input - 
        - df = Pandas DataFrame
        - columns = List input of all the columns which needs to be min-max scaled
    Expected Output -
        - df = Python DataFrame with Min-Max scaled attributes
        - scaler = Function which contains the scaling rules
    '''
    scaler = MinMaxScaler()
    data = pd.DataFrame(scaler.fit_transform(df.loc[:,columns]))
    data.index = df.index
    data.columns = df.columns
    return data, scaler

def label_encoder(df,columns):
    '''
    Function to label encode
    Required Input - 
        - df = Pandas DataFrame
        - columns = List input of all the columns which needs to be label encoded
    Expected Output -
        - df = Pandas DataFrame with lable encoded columns
        - le_dict = Dictionary of all the column and their label encoders
    '''
    le_dict = {}
    for c in columns:
        print("Label encoding column - {0}".format(c))
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values.astype('str')))
        df[c] = lbl.transform(list(df[c].values.astype('str')))
        #le_dict[c] = lbl
        le_dict[c] = dict(zip(lbl.classes_, lbl.transform(lbl.classes_)))
    return df, le_dict

def one_hot_encoder(df, columns):
    '''
    Function to do one-hot encoded
    Required Input - 
        - df = Pandas DataFrame
        - columns = List input of all the columns which needs to be one-hot encoded
    Expected Output -
        - df = Pandas DataFrame with one-hot encoded columns
    '''
    for each in columns:
        print("One-Hot encoding column - {0}".format(each))
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df.drop(columns,axis = 1)

## feature selection helper method

def pecor_selector(X, y,n=10):
    cor_list = []
    
    #Normalization: no
    #Impute missing values: yes
    
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-n:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in X.columns]
    #cor_support = [True if i in cor_feature]
    print(str(len(cor_feature)), 'selected features')
    return cor_support, cor_feature

def kbest_selector(X, y, n=10):
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import chi2
    from sklearn.preprocessing import MinMaxScaler
    
    #Normalization: MinMaxScaler (values should be bigger than 0)
    #Impute missing values: yes
    if n>len(X.columns):
        n = len(X.columns)
    X_norm = MinMaxScaler().fit_transform(X)
    chi_selector = SelectKBest(chi2, k=n)
    chi_selector.fit(X_norm, y)
    chi_support = chi_selector.get_support()
    chi_feature = X.loc[:,chi_support].columns.tolist()
    print(str(len(chi_feature)), 'selected features')
    return chi_support, chi_feature
    
def rfe_selector(X, y, n=18):    
    from sklearn.feature_selection import RFE
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import MinMaxScaler
     
    #Normalization: depend on the used model; yes for LR
    #Impute missing values: depend on the used model; yes for LR
    
    rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=n, step=1, verbose=5)
    X_norm = MinMaxScaler().fit_transform(X)
    rfe_selector.fit(X_norm, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:,rfe_support].columns.tolist()
    #rfe_feature = X.loc[:].columns.tolist()
    print(str(len(rfe_feature)), 'selected features')
    return rfe_support, rfe_feature
    
def rforest_selector(X, y, n=10):
    from sklearn.feature_selection import SelectFromModel
    from sklearn.ensemble import RandomForestClassifier
        
    #Normalization: No
    #Impute missing values: Yes
    
    if n>len(X.columns):
        n = len(X.columns)    
    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=50), threshold=10)
    embeded_rf_selector.fit(X, y)
    embeded_rf_support = embeded_rf_selector.get_support()
    #embeded_rf_support = [True if i in embeded_rf_selector.get_support() else False for i in X.columns]
    #embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
    embeded_rf_feature = X.iloc[:,embeded_rf_support.argsort()[::-1][:n]].columns.tolist()
    embeded_rf_support = [True if i in embeded_rf_feature else False for i in X.columns]
    print(str(len(embeded_rf_feature)), 'selected features')
    return embeded_rf_support, embeded_rf_feature


def select_features(X,y,n,rfe=0):    
	cor_support, cor_feature = pecor_selector(X,y,n)
	chi_support, chi_feature = kbest_selector(X,y,n)
	embeded_rf_support, embeded_rf_feature = rforest_selector(X,y,n)

	if rfe==1.0:
		rfe_support, rfe_feature = rfe_selector(X, y,n)
		feature_selection_df = pd.DataFrame({'Feature':X.columns, 'Chi-2':chi_support, 'RFE':rfe_support, 'Random Forest':embeded_rf_support})
		

	# put all selection together
	feature_selection_df = pd.DataFrame({'Feature':X.columns, 'pearson':cor_support , 'Chi-2':chi_support, 'Random Forest':embeded_rf_support})
	# count the selected times for each feature
	feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
	# display the top n
	feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
	feature_selection_df.index = range(1, len(feature_selection_df)+1)
	#feature_selection_df.index = range(len(feature_selection_df))
	return feature_selection_df

C:\Users\abhranshu\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\abhranshu\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\abhranshu\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\abhranshu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\abhranshu\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()


#### -- Complete leads dataset & carry forward features --

In [2]:
leads_feat_all = ['createdDate', 'Id','lead_id',
       'phone', 'Device_Type__c',  'Company__c',
       'Company', 'Designation__c',
       'Funded_By__c', 'Status', 'Site_Module__c',
       'LeadSource', 'Lead_Creation_mode__c',
       'Primary_Course_Interested__c', 'Primary_Category_of_Interest__c',
       'Primary_Training_Type__c', 'City__c', 'Country__c', 'Country',
       'Entry_Page__c', 'Lead_Owner_Email__c',
       'Lead Campaign',
       'sitemodule', 'lead_creation_mode', 'lead_entry_source', 'lead_source',
       'lead_medium', 'lead_campaign', 'lead_adgroup', 'lead_utm_term',
       'leadQueryType', 'leadTrainingType', 'leadGeo',
       'leadCity', 'leadCountry', 'querySourceString']

leads_feat_raw_only = ['createdDate', 'Id','lead_id',
       'phone', 'Device_Type__c',  'Company__c',
       'Company', 'Designation__c',
       'Funded_By__c', 'Status', 'Site_Module__c',
       'LeadSource', 'Lead_Creation_mode__c',
       'Primary_Course_Interested__c', 'Primary_Category_of_Interest__c',
       'Primary_Training_Type__c', 'City__c', 'Country__c', 'Country',
       'Entry_Page__c', 'Lead_Owner_Email__c']
  
leads_feat_enterprise_only = ['Lead Campaign','Id','lead_id',
       'sitemodule', 'lead_creation_mode', 'lead_entry_source', 'lead_source',
       'lead_medium', 'lead_campaign', 'lead_adgroup', 'lead_utm_term',
       'leadQueryType', 'leadTrainingType', 'leadGeo',
       'leadCity', 'leadCountry', 'querySourceString']

leads_feat_working01 = ['createdDate', 'Id','lead_id',
       'phone', 'Device_Type__c',  'Company__c','Country__c',
       'Company', 'Designation__c',
       'Funded_By__c', 'Status',
       'LeadSource', 'Lead_Creation_mode__c',
       'Primary_Course_Interested__c', 'Primary_Category_of_Interest__c',
       'Primary_Training_Type__c',
       'Lead_Owner_Email__c','lead_entry_source',
       'sitemodule', 'lead_creation_mode', 'lead_source',
       'lead_medium', 'lead_campaign', 'lead_adgroup', 'lead_utm_term',
       'leadQueryType', 'leadTrainingType', 'leadGeo',
       'leadCity', 'leadCountry', 'querySourceString']

leads_raw = pd.read_csv('C:/Users/abhranshu/Desktop/Incubating/Leads Scoring/leads_master_query_dump25nov18.csv',low_memory=True)
leads_raw = leads_raw[leads_feat_working01]
# keep only data from 2017 onwards
leads_raw = leads_raw.loc[pd.DatetimeIndex(leads_raw.createdDate).year>2015]
#pd.DatetimeIndex(leads_raw.createdDate).year.value_counts()

sf_opp_raw = pd.read_csv('C:/Users/abhranshu/Desktop/Incubating/Leads Scoring/opportunity_b_dump.csv')

sf_orders_raw = pd.read_csv('C:/Users/abhranshu/Desktop/Incubating/Leads Scoring/orders_b_dump.csv')
sf_orders_raw.payment_date__c = pd.to_datetime(sf_orders_raw.payment_date__c)

display(pd.DatetimeIndex(leads_raw.createdDate).year.value_counts())

leads_opp_concat = pd.merge(leads_raw,sf_opp_raw,how='left',left_on='lead_id',right_on='Id')
leads_opp_ord_concat = pd.merge(leads_opp_concat,sf_orders_raw,how='left',left_on='lead_id',right_on='Opportunity__c')

print("There are total %r obs in sales matx" %(len(leads_opp_ord_concat)))
print("There are %r obs where order are present in sales matx" %(len(leads_opp_ord_concat[~leads_opp_ord_concat.orderNumber.isnull()])))
print("There are %r obs where order are NOT present in sales matx" %(len(leads_opp_ord_concat[leads_opp_ord_concat.orderNumber.isnull()])))
print("***************************")
print("These obs will be part of sample set for our classification model !")

leads_opp_ord_concat.head(5)

2018    364608
2017    274238
2016    260695
Name: createdDate, dtype: int64

There are total 906715 obs in sales matx
There are 89712 obs where order are present in sales matx
There are 817003 obs where order are NOT present in sales matx
***************************
These obs will be part of sample set for our classification model !


,createdDate,Id_x,lead_id,phone,Device_Type__c,Company__c,Country__c,Company,Designation__c,Funded_By__c,Status,LeadSource,Lead_Creation_mode__c,Primary_Course_Interested__c,Primary_Category_of_Interest__c,Primary_Training_Type__c,Lead_Owner_Email__c,lead_entry_source,sitemodule,lead_creation_mode,lead_source,lead_medium,lead_campaign,lead_adgroup,lead_utm_term,leadQueryType,leadTrainingType,leadGeo,leadCity,leadCountry,querySourceString,Unnamed: 0_x,Id_y,AccountId,StageName,Closed_Date__c,Unnamed: 0_y,orderNumber,Opportunity__c,payment_date__c
0,2017-10-21 21:08:56,00Q0I00000rpjEVUAY,NaN,07982385527,desktop,NaN,India,NaN,NaN,NaN,Inactive,SL-Website,Online,Data Science with Python,Data Science & Business Intelligence,Online Classroom Flexi-Pass,shikha.mishra@simplilearn.net,SL-Website,chat_triggered,online,google,organic,(organic),NaN,(not provided),B2C,LVC,IN,NaN,India,https://www.simplilearn.com/big-data-and-analy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,2017-10-22 05:05:08,00Q0I00000rpjT1UAI,0060I00000XZw2BQAT,7126160,desktop,Anse la Raye RC Infant School,Saint Lucia,NaN,Principal,Self,Converted,SL-Website,Online,Digital Marketing Specialist,Digital Marketing,Online Classroom Flexi-Pass,sajith.cs@simplilearn.net,SL-Website,course preview,online,google,cpc,search-brand-row,NaN,simpli learn,B2C,LVC,AMERICAS,NaN,Saint Lucia,https://www.simplilearn.com/digital-marketing/...,81270.0,0060I00000XZw2BQAT,0010I00001b8cd3QAA,Closed Won,2018-09-20,30938.0,CSTM_67V0YZ53NRB,0060I00000XZw2BQAT,2018-09-21
2,2017-10-22 05:10:19,00Q0I00000rpjTQUAY,NaN,NaN,NaN,NaN,United States,NaN,NaN,NaN,Inactive,SL-Website,Online,Agile and Scrum Career Advancement Bundle,Agile and Scrum,Online Classroom Flexi-Pass,sarath@simplilearn.net,SL-Website,chat_triggered,online,(direct),(none),(direct),NaN,NaN,B2C,LVC,AMERICAS,Atlanta,United States,https://www.simplilearn.com/agile-and-scrum/pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2017-10-22 07:10:18,00Q0I00000rpjaCUAQ,NaN,9969593909,desktop,NaN,India,NaN,NaN,NaN,Inactive,SL-Website,Online,Data Scientist,Data Science & Business Intelligence,Online Classroom Flexi-Pass,mousumi.das@simplilearn.net,SL-Website,bundle agenda,online,accounts.simplilearn.com,referral,(referral),NaN,NaN,B2C,LVC,IN,Mumbai,India,https://www.simplilearn.com/big-data-and-analy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2017-10-22 11:05:07,00Q0I00000rpjkvUAA,NaN,09444202015,desktop,TCS,India,NaN,.,Self,Inactive,SL-Website,Online,Data Scientist,Data Science & Business Intelligence,Online Classroom Flexi-Pass,sharath.bazar@simplilearn.net,SL-Website,bundle agenda,online,google,cpc,search-bigdata-ds-low-exact-in-adgroup-data-sc...,data science,data scientist,B2C,LVC,IN,Chetput,India,https://www.simplilearn.com/big-data-and-analy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


#### -- Define Business Rules/ Feature Engineering --
    - feature created: converted(boolean) for target variable of leads with/wo orders. 
    - feature created: countries_cat for top-leads & low-leads countries.
    - feature created: sitemodule_cat for PSM & NON-PSM Site Modules.
    - feature created: days_since_payment for elapsed payment date for leads.
    - feature created: channel for leads channels.
    - feature created: month_created_date for lead created date month.
    - feature created: sitemodule_cluster for CPCABA+Chat leads.
    - removing features which are low value addition to model (manual).

In [3]:
# Pass features from here:-
pre_eng_features = [ 'Status','Primary_Course_Interested__c', 'Country__c', 'sitemodule',
   'lead_campaign','lead_source','lead_entry_source','lead_medium','createdDate','lead_creation_mode',
   'leadQueryType',  'leadTrainingType', 'leadGeo',
   'payment_date__c','converted']

In [17]:
# Set All Business Rules Here :- DUPLICATE !!!

from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

############### Write code below ##################

leads_opp_concat = pd.merge(leads_raw,sf_opp_raw,how='left',left_on='lead_id',right_on='Id')
leads_opp_ord_concat = pd.merge(leads_opp_concat,sf_orders_raw,how='left',left_on='lead_id',right_on='Opportunity__c')

leads_wo_orders_subset = leads_opp_ord_concat.loc[leads_opp_ord_concat.payment_date__c.isnull()][:70700]
leads_w_orders_subset = leads_opp_ord_concat.loc[~leads_opp_ord_concat.payment_date__c.isnull()][:61477]
leads_opp_ord_concat = pd.concat([leads_wo_orders_subset,leads_w_orders_subset])

# Set target variable as Converted=1/0
leads_opp_ord_concat.loc[~leads_opp_ord_concat.orderNumber.isnull(),'converted'] = 1
leads_opp_ord_concat.loc[leads_opp_ord_concat.orderNumber.isnull(),'converted'] = 0


target_var = 'converted'
sales_matx = leads_opp_ord_concat[pre_eng_features]
sales_matx.drop([],axis=1,inplace=True)

## top-leads & low-leads countries
country_list = sales_matx.Country__c.value_counts().sort_values(ascending=False).index
top_leads_countries = country_list[0:10]
low_leads_countries = country_list[10:]
len(top_leads_countries)

sales_matx.loc[sales_matx.Country__c.isin(top_leads_countries),'countries_cat']= 'top_leads_country'
sales_matx.loc[sales_matx.Country__c.isin(low_leads_countries),'countries_cat']= 'low_leads_country'

sales_matx['countries_cat'].value_counts()

## PSM & NON-PSM Site Modules
psm_list =['query box','toll','phone','request a call','na-payment','course agenda','bundle agenda','b2c masters','category_query_box','free_trial','free trial cancelled','previewobj','course preview','chat','chat_initiated','chat_triggered']

sales_matx.loc[sales_matx.sitemodule.isin(psm_list),'sitemodule_psm']= 1
sales_matx.loc[~sales_matx.sitemodule.isin(psm_list),'sitemodule_psm']= 0


## Age of lead from Payment Date
from datetime import datetime
def compare_dates(date):
    date_format = '%Y/%m/%d'
    #current_date = datetime.strptime(date, date_format)
    current_date = pd.to_datetime(date)
    today = datetime.today()
    diff = today - current_date
    return diff.days

#apply this function to your pandas dataframe to get days elapsed difference
sales_matx['days_since_payment'] = sales_matx['payment_date__c'].apply(compare_dates)

#sales_matx['days_since_leadclosed'] = sales_matx['Closed_Date__c'].apply(compare_dates)

# Creating channel variable
sales_matx.loc[sales_matx['lead_creation_mode'] == 'online', 'channel']= "online-others"
sales_matx.loc[sales_matx['lead_source'].str.contains("direct|(direct)",na=False), 'channel']= "direct"
sales_matx.loc[sales_matx['sitemodule'].str.contains("tollfree",na=False), 'channel']= "tollfree"
sales_matx.loc[sales_matx['lead_source'].str.contains("steelhouse",na=False), 'channel']= "Steelhouse"
sales_matx.loc[(sales_matx['lead_source'].str.contains("facebook",na=False)) & (sales_matx['lead_medium'].str.contains("rtg|dpa",na=False)), 'channel']= "facebook-rtg"
sales_matx.loc[sales_matx['lead_source'].str.contains("quora-cpc",na=False), 'channel']= "quora-cpc"
sales_matx.loc[sales_matx['lead_source'].str.contains("quora-referral",na=False), 'channel']= "quora-referral"
sales_matx.loc[sales_matx['lead_source'].str.contains("quora-rtg",na=False), 'channel']= "quora-rtg"
sales_matx.loc[sales_matx['lead_source'].str.contains("quora-rtg",na=False), 'channel']= "quora-rtg"
sales_matx.loc[(sales_matx['lead_source'].str.contains("google",na=False)) & (sales_matx['lead_medium'].str.contains("cpc",na=False))& (sales_matx['lead_campaign'].str.contains("youtube",na=False)), 'channel']= "youtube-cpc"
sales_matx.loc[(sales_matx['lead_source'].str.contains("youtube",na=False)) & (sales_matx['lead_medium'].str.contains("cpc",na=False)), 'channel']= "youtube-cpc"
sales_matx.loc[(sales_matx['lead_source'].str.contains("google",na=False)) & (sales_matx['lead_campaign'].str.contains("display",na=False)), 'channel']= "google-cpc-display"
sales_matx.loc[(sales_matx['lead_source'].str.contains("google",na=False)) & (sales_matx['lead_campaign'].str.contains("search-brand",na=False)), 'channel']= "google-cpc-brand"
sales_matx.loc[(sales_matx['lead_source'].str.contains("utmgclid|google",na=False))& (sales_matx['lead_medium'].str.contains("cpc",na=False)) & (sales_matx['lead_campaign'].str.contains("atp",na=False)), 'channel']= "google-cpc-partner"
sales_matx.loc[(sales_matx['lead_source'].str.contains("utmgclid|google",na=False)) & (sales_matx['lead_medium'].str.contains("cpc",na=False)), 'channel']= "google-cpc"
sales_matx.loc[(sales_matx['lead_source'].str.contains("bing",na=False)) & (sales_matx['lead_campaign'].str.contains("brand",na=False)), 'channel']= "bing-brand"
sales_matx.loc[sales_matx['lead_source'].str.contains("monster|naukri|dice|jobsahead|bayt|shine|iimjobs|indeed|hirist|leadgen",na=False), 'channel']= 'lead-gen'
sales_matx.loc[sales_matx['lead_source'].str.contains("criteo",na=False), 'channel']= "criteo"
sales_matx.loc[sales_matx['lead_medium'].str.contains("organic",na=False), 'channel']= "organic"
sales_matx.loc[(sales_matx['lead_source'].str.contains("Drip",na=False)) & (sales_matx['lead_medium'].str.contains("mcc|ebook|actonc",na=False)), 'channel']= "drip-cold"
sales_matx.loc[sales_matx['lead_source'].str.contains("Drip",na=False), 'channel']= "drip"
sales_matx.loc[(sales_matx['lead_source'].str.contains("contentmarketing",na=False)) | (sales_matx['lead_medium'].str.contains("contentmarketing",na=False)) | (sales_matx['lead_campaign'].str.contains("contentmarketing",na=False)), 'channel']= "content-marketing"
sales_matx.loc[sales_matx['lead_medium'].str.contains("referral",na=False), 'channel']= "referral"
sales_matx.loc[(sales_matx['lead_source'].str.contains("linkedin",na=False)) & (sales_matx['lead_medium'].str.contains("b2b|b2b-visit|b2b-leadgen",na=False)), 'channel']= "linkedin-b2b"
sales_matx.loc[(sales_matx['lead_source'].str.contains("linkedin",na=False)) & (sales_matx['lead_medium'].str.contains("atp-leadgen|partner-visit|partner-leadgen",na=False)), 'channel']= "linkedin-partner"
sales_matx.loc[(sales_matx['lead_source'].str.contains("linkedin",na=False)) & (sales_matx['lead_medium'].str.contains("leadgen|linkedin-cpc|b2c-visit|b2c-leadgen",na=False)), 'channel']= "linkedin-b2c"
sales_matx.loc[(sales_matx['lead_source'].str.contains("shareasale|payoom|komli|cj|omg|dgm|affiliate-cpa",na=False)) | (sales_matx['lead_medium'].str.contains("affiliate-cpa",na=False)), 'channel']= "affiliate-networks"
sales_matx.loc[sales_matx['lead_source'].str.contains("adroll",na=False), 'channel']= "adroll"
sales_matx.loc[~(sales_matx['lead_medium'].str.contains("social",na=False)) & ((sales_matx['lead_source'].str.contains("facebook|facebook_leadgen",na=False)) & (sales_matx['lead_campaign'].str.contains("marketing_facebook",na=False))), 'channel']= "facebook-b2c"
sales_matx.loc[(sales_matx['lead_source'].str.contains("facebook",na=False)) & ((sales_matx['lead_medium'].str.contains("b2c-visit|b2c-leadgen",na=False))), 'channel']= "facebook-b2c"
sales_matx.loc[sales_matx['lead_source'].str.contains("social",na=False), 'channel']= "social"
sales_matx.loc[(sales_matx['lead_source'].str.contains("affiliate",na=False)) & ((sales_matx['lead_medium'].str.contains("affiliate-cpl|affiliate-cpm",na=False))), 'channel']= "media-buying"
sales_matx.loc[sales_matx['lead_source'].str.contains("bizo",na=False), 'channel']= "bizo"
sales_matx.loc[sales_matx['lead_source'].str.contains("adelement",na=False), 'channel']= "adelement"
sales_matx.loc[sales_matx['lead_source'].str.contains("twitter",na=False), 'channel']= "twitter"
sales_matx.loc[sales_matx['lead_source'].str.contains("youtube",na=False), 'channel']= "youtube"
sales_matx.loc[(sales_matx['lead_source'].str.contains("mail",na=False)) | (sales_matx['lead_medium'].str.contains("mail",na=False)), 'channel']= "sales-email"
sales_matx.loc[(sales_matx['lead_source'].str.contains("bing",na=False)) & (sales_matx['lead_medium'].str.contains("cpc",na=False)), 'channel']= "sales-email"
sales_matx.loc[sales_matx['lead_medium'].str.contains("social",na=False), 'channel']= "social"
sales_matx.loc[sales_matx['lead_creation_mode'].str.contains("offline",na=False), 'channel']= sales_matx['lead_entry_source']
sales_matx.loc[sales_matx['channel'].isnull(),'channel'] = "Other"
#sales_matx.channel.value_counts()


# extracting month from created date
sales_matx['created_date_conv'] = pd.to_datetime(sales_matx['createdDate'], format = '%Y-%m-%d %H:%M:%S')
sales_matx['month_created_date'] = sales_matx.created_date_conv.dt.month

# creating site module cluster
sales_matx.loc[sales_matx['sitemodule'].str.contains("course agenda|course preview",na=False), 'sitemodule_cluster']= 'Course Preview/ Agenda'
sales_matx.loc[sales_matx['sitemodule'].str.contains("chat|call",na=False), 'sitemodule_cluster']= 'chat/call'
sales_matx.loc[sales_matx['sitemodule_cluster'].isnull(),'sitemodule_cluster'] = "Other"


"""pat1 = r'PMP'
pat2 = r'ITIL'
pat3 = r'PRINCE2'
pat4 = r'Scrum'
pat5 = r'Big%Data|Hadoop|Spark'
pat6 = r'AWS'
pat7 = r'Azure'
pat8 = r'Marketing'
pat9 = r'Data%Science|Machine%learning|ML|Science|AI|Artificial|R%Programming'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat1),'product_name'] = 'PMP'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat2),'product_name'] = 'ITIL'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat3),'product_name'] = 'PRINCE2'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat4),'product_name'] = 'CSM'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat5),'product_name'] = 'Hadoop'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat6),'product_name'] = 'AWS-Cloud'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat7),'product_name'] = 'Azure-Cloud'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat8),'product_name'] = 'Digital-Marketing'
sales_matx.loc[sales_matx.product_name__c.str.contains(pat9),'product_name'] = 'AWS-Cloud'
sales_matx[:5]"""

## Top_10_countries_based_on_leads_generating_PSM
var_10_psm_country = sales_matx.groupby(['Country__c','sitemodule_psm']).size().to_frame('count').reset_index().sort_values(['count'],ascending=False)
fet_top_psm_countries = var_10_psm_country[var_10_psm_country.sitemodule_psm==1]['Country__c'][:10]
fet_botm_psm_countries = var_10_psm_country[var_10_psm_country.sitemodule_psm==1]['Country__c'][10:]
top_psm_countries_vals = list(fet_top_psm_countries.values)
sales_matx.loc[sales_matx.Country__c.isin(top_psm_countries_vals),'top_psm_country'] = 1
sales_matx.loc[~sales_matx.Country__c.isin(top_psm_countries_vals),'top_psm_country'] = 0

## Top_10_sitemodules_based_on_orders
var_leads_w_ords=sales_matx[~sales_matx.payment_date__c.isnull()]
var_top_converted_sm = var_leads_w_ords.groupby(['sitemodule']).size().to_frame('count').reset_index().sort_values(['count'],ascending=False)
fet_top_converted_sm = var_top_converted_sm['sitemodule'][:10]
fet_bottm_converted_sm = var_top_converted_sm['sitemodule'][10:]
top_converted_sm_vals = list(fet_top_converted_sm.values)
bottm_converted_sm_vals = list(fet_bottm_converted_sm.values)
sales_matx.loc[sales_matx.sitemodule.isin(top_converted_sm_vals),'top_converted_sm'] = 1
sales_matx.loc[sales_matx.sitemodule.isin(bottm_converted_sm_vals),'top_converted_sm'] = 0

# top 10 channel based on lead gen
var_10_channel = sales_matx.groupby(['channel']).size().to_frame('count').reset_index().sort_values(['count'],ascending=False)
fet_top_channel = var_10_channel['channel'][:10]
top_channel_vals = list(fet_top_channel.values)
sales_matx.loc[sales_matx.channel.isin(top_channel_vals),'top_channel_flag'] = 1
sales_matx.loc[~sales_matx.channel.isin(top_channel_vals),'top_channel_flag'] = 0

def t(row):
    if row['top_channel_flag'] == 1:
        val = row['channel']
    else:
        val = 'Other'
    return val
sales_matx['Top_Channel'] = sales_matx.apply(t, axis=1)

sales_matx = sales_matx.loc[sales_matx.leadQueryType=='B2C']

## Top_10_countries_based_on_leads_generating_PSM - variable

var_10_psm_country = sales_matx.groupby(['Country__c','sitemodule_psm']).size().to_frame('count').reset_index().sort_values(['count'],ascending=False)
fet_top_psm_countries = var_10_psm_country[var_10_psm_country.sitemodule_psm==1]['Country__c'][:10]
fet_botm_psm_countries = var_10_psm_country[var_10_psm_country.sitemodule_psm==1]['Country__c'][10:]
top_psm_countries_vals = list(fet_top_psm_countries.values)
sales_matx.loc[sales_matx.Country__c.isin(top_psm_countries_vals),'top_psm_country'] = 1
sales_matx.loc[~sales_matx.Country__c.isin(top_psm_countries_vals),'top_psm_country'] = 0

#sales_matx.top_psm_country.unique()
def f(row):
    if row['top_psm_country'] == 1:
        val = row['Country__c']
    else:
        val = 'Other'
    return val
sales_matx['Top_psm_country_lead_gen'] = sales_matx.apply(f, axis=1)


def combine_features(df,fet1,fet2):
    prefx = df[fet1].name+df[fet2].name
    sufx = '_comb'
    for item1 in df[fet1].unique():
        for item2 in df[fet2].unique():
            df.loc[(df[fet1]==item1) & (df[fet2]==item2),(prefx+sufx)]=item1+'-'+str(item2)

combine_features(sales_matx,'sitemodule_cluster','leadGeo')
combine_features(sales_matx,'channel','leadGeo')

display(sales_matx.apply(pd.Series.nunique).sort_values(ascending=False))

C:\Users\abhranshu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:75: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


created_date_conv                 115437
createdDate                       115437
lead_campaign                       7041
payment_date__c                      985
days_since_payment                   985
lead_source                          701
Primary_Course_Interested__c         645
lead_medium                          297
Country__c                           209
channelleadGeo_comb                  143
sitemodule                            93
lead_entry_source                     83
channel                               57
sitemodule_clusterleadGeo_comb        14
month_created_date                    12
Top_Channel                           11
Top_psm_country_lead_gen              11
Status                                 9
leadGeo                                6
leadTrainingType                       3
sitemodule_cluster                     3
lead_creation_mode                     2
countries_cat                          2
sitemodule_psm                         2
top_psm_country 

In [18]:
# drop redundant/less_imp fields

sales_matx_copy = sales_matx.copy()
sales_matx_temp = sales_matx.copy()

def drop_features(df,lst):
    return df.drop(lst,axis=1,inplace=True)

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def col_selector(df,keep_cols,use=False):
    if use==True:
        mandate_cols = []        
      
        rem_cols = diff(df.columns,keep_cols)
        rem_cols.remove('converted')
        
        drop_features(df,rem_cols)
    else: 
        print("you have not opted for column selector functionality!!")
        
        return df
    
def filter_method_fselection(df,how_many=5,sel_criteria='cutoff',use=False):
    if use==True:
        print("Called after feature engineering !!")
        print("If column selector is used, this method is called after that !!")
        print("some methods like pearson, rf require encoded data ??")
        X_lab_enc,X_lab_enc_dict = label_encoder(df,df.columns)
        feature_selection_df = select_features(X_lab_enc.loc[:,X_lab_enc.columns!='converted'],X_lab_enc['converted'],how_many)
        if sel_criteria == 'max':
            f_methd_list = feature_selection_df.loc[feature_selection_df['Total']==max(feature_selection_df['Total'])]['Feature'].tolist()
        elif sel_criteria == 'cutoff':
            f_methd_list = feature_selection_df.loc[feature_selection_df['Total']>1.0]['Feature'].tolist()
    return f_methd_list
    
"""
rem_feats = ['Primary_Course_Interested__c',  'top_channel_flag','lead_campaign','lead_source','lead_medium','createdDate',
             'created_date_conv','days_since_payment','payment_date__c','month_created_date','top_psm_country','top_converted_sm','lead_creation_mode'
             ,'sitemodule_psm','channelleadGeo_comb','sitemodule_clusterleadGeo_comb','Top_psm_country_lead_gen','Top_Channel','Status'
             ,'countries_cat','leadQueryType']

drop_feats(sales_matx_copy,rem_feats)
post_eng_selected_features = list(sales_matx_copy.columns)"""

display(sales_matx_copy.apply(pd.Series.nunique).sort_values(ascending=False))

created_date_conv                 115437
createdDate                       115437
lead_campaign                       7041
payment_date__c                      985
days_since_payment                   985
lead_source                          701
Primary_Course_Interested__c         645
lead_medium                          297
Country__c                           209
channelleadGeo_comb                  143
sitemodule                            93
lead_entry_source                     83
channel                               57
sitemodule_clusterleadGeo_comb        14
month_created_date                    12
Top_Channel                           11
Top_psm_country_lead_gen              11
Status                                 9
leadGeo                                6
leadTrainingType                       3
sitemodule_cluster                     3
lead_creation_mode                     2
countries_cat                          2
sitemodule_psm                         2
top_psm_country 

In [19]:
# imputation of missing values

col_selector(sales_matx_copy,['Country__c', 'sitemodule', 'leadGeo','channel'],True)
col_selector(sales_matx_temp,['Country__c', 'sitemodule', 'leadGeo','channel'],True)

fets_selected_f_filter_methd = filter_method_fselection(sales_matx_temp,3,'cutoff',True)
rem_cols = diff(sales_matx_copy.columns,fets_selected_f_filter_methd)
rem_cols.remove('converted')
drop_features(sales_matx_copy,rem_cols)

sales_matx_imputed = treat_missing_categorical(sales_matx_copy,fets_selected_f_filter_methd,how = 'mode')

"""only_OHE_these = [
 'leadQueryType',
 'leadTrainingType',
 'leadGeo',
 'Top_psm_country_lead_gen',
 'Top_Channel',
 'Status',
 'countries_cat',
 'channel',
 'sitemodule_cluster',
 'sitemodule_clusterleadGeo_comb',
 'channelleadGeo_comb']
 
 only_OHE_these = [
 'channel',
 'sitemodule',
 'leadTrainingType',
 'leadGeo']
"""

only_OHE_these = fets_selected_f_filter_methd

# OHE- one hot enconding of features
#fet = post_eng_selected_features.copy()
#fet.remove('converted')
OHE_enc = one_hot_encoder(sales_matx_imputed,only_OHE_these)
X_enc = OHE_enc

Called after feature engineering !!
If column selector is used, this method is called after that !!
some methods like pearson, rf require encoded data ??
Label encoding column - Country__c
Label encoding column - sitemodule
Label encoding column - leadGeo
Label encoding column - converted
Label encoding column - channel
3 selected features
3 selected features
3 selected features
Filling missing values with mode for columns - sitemodule
Filling missing values with mode for columns - leadGeo
Filling missing values with mode for columns - Country__c
One-Hot encoding column - sitemodule
One-Hot encoding column - leadGeo
One-Hot encoding column - Country__c


#### -- create training and validation set -- 

#### -- Fit the enconded dataset to a Random Forest Model --

In [25]:
# use feature importance for feature selection

from numpy import sort
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
# load data

X, y = X_enc.loc[:,X_enc.columns!='converted'], X_enc[target_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
# fit model on all training data
#model = XGBClassifier()
model = GradientBoostingClassifier(random_state=0, max_depth=3)
model.fit(X_train, y_train)
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
	# select features using threshold
	selection = SelectFromModel(model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = GradientBoostingClassifier()
	selection_model.fit(select_X_train, y_train)
	# eval model
	select_X_test = selection.transform(X_test)
	y_pred = selection_model.predict(select_X_test)
	predictions = [round(value) for value in y_pred]
	accuracy = accuracy_score(y_test, predictions)
	print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%
Thresh=0.000, n=308, Accuracy: 68.19%


KeyboardInterrupt: 

In [44]:
sorted(thresholds[thresholds>0.0],ascending=False)[:5]

TypeError: 'ascending' is an invalid keyword argument for this function

In [ ]:
from sklearn.feature_selection import SelectFromModel

X, y = X_enc.loc[:,X_enc.columns!='converted'], X_enc[target_var]
X_train_dtm, X_test_dtm, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

print(color.BOLD + color.BLUE + "train dataset shape: %s , test dataset shape:%s "%(X_train_dtm.shape,X_test_dtm.shape))

#regr2 = SelectFromModel(RandomForestClassifier(n_estimators=n,max_features=10), threshold='1.25*median')
#rf = RandomForestClassifier(max_depth=4,n_estimators=80, random_state=0,criterion='entropy')
#rf = RandomForestClassifier(max_depth=15,n_estimators=200, random_state=0,max_features=7,min_samples_split=2)
rf = GradientBoostingClassifier(random_state=0, max_depth=4)

rf.fit(X_train_dtm,y_train)
y_pred_class = rf.predict(X_test_dtm)

print("*****************************************")
print("*****************************************")
print(color.BOLD + color.BLUE + "Accuracy obtained from model is :%.2f%% "%(metrics.accuracy_score(y_test, y_pred_class)*100))
print("*****************************************")
print("*****************************************")

# Get numerical feature importances
importances = list(rf.feature_importances_)

#features= selected_features
fet_cols = X_enc.loc[:,X_enc.columns!='converted'].columns
features= fet_cols

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:15]] 
print(color.BOLD + color.BLUE + "         ******           ")

from sklearn.metrics import classification_report, roc_curve

print (classification_report(y_test, rf.predict(X_test_dtm)))
#fpr, tpr, thresholds = roc_curve(y_test, regr2.predict_proba(X_test_dtm), pos_label=3)
print(color.BOLD + color.BLUE + "         ******           ")

#f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

feat_importances = pd.Series(rf.feature_importances_, index=fet_cols)
feat_importances.nlargest(10).plot(kind='bar')

test_score_matx = X_test_dtm.copy()
test_score_matx['score'] = np.round(rf.predict_proba(test_score_matx)[:,1]*100,2)

print("*****************************************")
print("*****************************************")
test_score_matx = X_test_dtm.copy()
test_score_matx['score'] = np.round(rf.predict_proba(test_score_matx)[:,1]*100,2)
print("Maximum predicted score  is: %r"%(max(test_score_matx['score'])))
print("*****************************************")
print("*****************************************")

# info of the highest score lead
result = test_score_matx.loc[test_score_matx['score']==max(test_score_matx['score'])]
k = [i[0] for i in feature_importances[:15]]
k.append('score')
debug_result_df = result[k]
result.loc[result.index[:1]][k]

In [ ]:
result = test_score_matx.loc[test_score_matx['score']>80.0]
k = [i[0] for i in feature_importances[:15]]
k.append('score')
sales_matx.loc[sales_matx.index.isin(result.index)][:3]

In [ ]:
pd.crosstab(index=sales_matx_copy['leadGeo'], columns=sales_matx_copy['converted'],normalize='all',
                        margins=True, margins_name="Total")

In [ ]:
display(len(sales_matx_copy.loc[(sales_matx_copy.converted==1.0) & (sales_matx_copy.sitemodule=='Converted')])/len(sales_matx_copy)*100)

In [ ]:
for colo in sales_matx_copy.columns:
    if ((colo!='converted') & (len(sales_matx[colo].unique())<10)):
        f, ax = plt.subplots(figsize=(4,4))
        sns.heatmap(pd.crosstab(index=sales_matx_copy[colo], columns=sales_matx_copy['converted'],normalize='all',
                        margins=True, margins_name="Total"),cmap="YlGnBu", annot=True, cbar=False)

In [ ]:
display(len(sales_matx.loc[(sales_matx.leadTrainingType=='LVC')])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.leadTrainingType=='LVC') & (sales_matx.converted==1.0)])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.leadTrainingType=='CR')])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.leadTrainingType=='CR') & (sales_matx.converted==1.0)])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.leadTrainingType=='OSL')])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.leadTrainingType=='OSL') & (sales_matx.converted==1.0)])/len(sales_matx)*100)

In [ ]:
display(len(sales_matx.loc[(sales_matx.channel=='offline') & (sales_matx.leadGeo=='AMERICAS')])/len(sales_matx)*100)

In [ ]:
display(len(sales_matx.loc[(sales_matx.converted==1.0) & (sales_matx.sitemodule_cluster=='Course Preview/ Agenda')])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.converted==1.0) & (sales_matx.sitemodule_cluster=='chat/call')])/len(sales_matx)*100)
display(len(sales_matx.loc[(sales_matx.converted==1.0) & (sales_matx.sitemodule_cluster=='Other')])/len(sales_matx)*100)

In [ ]:
## ** plotting the cumulative importance line ** 

# list of x locations for plotting
x_values = list(range(len(importances)-98))

# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances[:10]]
sorted_features = [importance[0] for importance in feature_importances[:10]]

# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)

# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')

# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')

# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')

# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:
#test_score_matx['score'].value_counts()

In [ ]:
#result = test_score_matx.loc[test_score_matx['score']==max(test_score_matx['score'])]
result = test_score_matx.loc[test_score_matx['score']==74.68]
k = [i[0] for i in feature_importances[:15]]
k.append('score')
sales_matx.loc[sales_matx.index.isin(result.index) & (sales_matx.converted==0)][:3]

In [ ]:
result.loc[result.index[:1]][k]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

orig = np.array([6, 9, 8, 2, 5, 4, 5, 3, 3, 6])

ohe = OneHotEncoder()
encoded = ohe.fit_transform(orig.reshape(-1, 1)) # input needs to be column-wise

decoded = encoded.dot(ohe.active_features_).astype(int)
#print(np.allclose(orig, decoded))
decoded

In [ ]:
from treeinterpreter import treeinterpreter as ti

instances = X_test_dtm.loc[X_test_dtm.index==632699]

print ("Instance 0 prediction:", rf.predict(instances[:1]))
#print ("Instance 1 prediction:", rf.predict(instances[1:2]))

prediction, bias, contributions = ti.predict(rf, instances)

for i in range(len(instances)):
    print ("Instance", i)
    print ("Bias (trainset mean)", bias[i])
    print ("Feature contributions:")
    for c, feature in zip(contributions[i],X_enc.columns):
        print (feature, c)
    print ("-"*20)

In [ ]:
sales_matx.columns

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)
            
cs = ColumnSelector(columns=["Status"])
cs.fit_transform(sales_matx).head()

In [ ]:
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, Imputer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])
    
preprocess_pipeline = make_pipeline(
    ColumnSelector(columns=x_cols),
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            TypeSelector(np.number),
            Imputer(strategy="median"),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            TypeSelector("category"),
            Imputer(strategy="most_frequent"),
            OneHotEncoder()
        )),
        ("boolean_features", make_pipeline(
            TypeSelector("bool"),
            Imputer(strategy="most_frequent")
        ))
    ])
)    
    
ts = TypeSelector("category")
ts.fit_transform(X).head()    

In [ ]:
X_train[:3]

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, Imputer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC
from sklearn.decomposition import NMF

import numpy as np
import pandas as pd

#import pmlb

import random

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

sns.set(rc={"figure.figsize": (12, 8)})

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)


class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])
    
class Onehotenc(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype

    def fit(self, X, y=None):
        for each in X.columns:
            print("One-Hot encoding column - {0}".format(each))
            dummies = pd.get_dummies(X[each], prefix=each, drop_first=False)
            X = pd.concat([X, dummies], axis=1)
        return X.drop(columns,axis = 1)

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X    
    
#df = pmlb.fetch_data('churn', return_X_y=False)
df = sales_matx

# Remove the target column and the phone number
x_cols = [c for c in df if c not in ["converted"]]

binary_features = ['sitemodule_psm','top_psm_country','top_converted_sm','top_channel_flag']

catg = []
for c in sales_matx.select_dtypes(include=['object']):
    catg.append(c)   
categorical_features = catg

# Column types are defaulted to floats
X = df
#(
    #df
    #.drop(["converted"], axis=1)
    #.astype(float)
#)
X[binary_features] = X[binary_features].astype("bool")

# Categorical features can't be set all at once
for f in categorical_features:
    X[f] = X[f].astype("category")

y = df.converted

# Partition data set into training/test split (2 to 1 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)


preprocess_pipeline = make_pipeline(
    ColumnSelector(columns=x_cols),
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            TypeSelector(np.number),
            Imputer(strategy="median"),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            TypeSelector("category"),
            Imputer(strategy="most_frequent"),
            OneHotEncoder()
        )),
        ("boolean_features", make_pipeline(
            TypeSelector("bool"),
            Imputer(strategy="most_frequent")
        ))
    ])
)

pipe = Pipeline(
    [        
            ("ohe",Onehotenc("category")),
            ("debug", Debug()),
            ("nmf", NMF())
        
    ])

classifier_pipeline = make_pipeline(
    preprocess_pipeline,
    SVC(kernel="rbf", random_state=42)
)

param_grid = {
    "svc__gamma": [0.1 * x for x in range(1, 2)]
}

classifier_model = GridSearchCV(classifier_pipeline, param_grid, cv=3)
#classifier_model.fit(X_train, y_train)
X_train[:3]
pipe.fit_transform(X_train)

In [ ]:
def df_score_gen(df,fet_imp=""):

    var_lead = 1
    var_sm = 1
    var_p_age = 1
    var_cp = 1
    var_ctype = 1
        
    if (df['Region'] == 'IN'):
        var_lead = var_lead*50
    elif df['Region'] == "AMERICAS":
        var_lead = var_lead*30
    elif df['Region'] == "ROW":
        var_lead = var_lead*20
    
    if df['sitemodule_type'] == "PSM":
        var_sm = var_sm*50
    elif df['sitemodule_type'] == "NON-PSM":
        var_sm = var_sm*20
        
    if df['days_since_payment'] <= 273:
        var_p_age = var_p_age*50
    elif df['days_since_payment'] > 273:
        var_p_age = var_p_age*20 
        
    if df['CourseProgress'] >= 75.0:
        var_cp = var_cp*50
    elif df['CourseProgress'] < 75.0:
        var_cp = var_cp*20  
        
    if df['course_type'] == "high_selling":
        var_ctype = var_ctype*50
    elif df['course_type'] == "low_selling":
        var_ctype = var_ctype*20     
        
    #return var_lead*fet_imp['leadGeo']+var_sm*fet_imp['Site_Module__c']
    return var_p_age*0.65+var_ctype*0.28+var_sm*0.05+var_lead*0.02
    
combined_working_set['lead_score'] = combined_working_set.apply(df_score_gen, axis = 1) 